In [1]:
api_key = "pcsk_GTuk4_uTDkuarxCHrTycKAxtqb5Gb3bZ9sueu4g6yrxCgk2sfrQK28kn5GVQW9AVtdFg"

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [3]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
# initialize the Pinecone client
pc = Pinecone(api_key=api_key)

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimension of dense vector
        metric="dotproduct", # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [4]:
index = pc.Index(index_name)
index

In [7]:
# vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default() # tfidf encoder for sparse representation
bm25_encoder

In [11]:
# import nltk
# nltk.download('punkt_tab')

sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
]

# applying tfidf values on these sentence
bm25_encoder.fit(sentences)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 55.03it/s]


In [12]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [13]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x300a603e0>, index=<pinecone.data.index.Index object at 0x1077dc170>)

In [14]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:03<00:00,  3.68s/it]


In [16]:
retriever.invoke("Which city i visited first ?") 

[Document(metadata={'score': 0.248462215}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.241951913}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.239908785}, page_content='In 2023, I visited Paris')]